In [1]:
from pytdx.config.hosts import hq_hosts
import pandas as pd
from pytdx.hq import TdxHq_API
import seaborn as sns

from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import time

In [2]:
stocks_hyao = [(1, '600482','中国动力'), 
                (1, '588000','科创50ETF'), 
                (0, '300308','中际旭创'), 
                (1, '688777','中控技术')]

stocks_lan = [(1, '603019','中科曙光'),]

In [3]:
def bollinger_bands(df, n, m):
    # takes dataframe on input
    # n = smoothing length
    # m = number of standard deviations away from MA
    
    #typical price
    TP = (df['high'] + df['low'] + df['close']) / 3
    # but we will use Adj close instead for now, depends
    
    data = df['close']
    #data = TP
    #data = df['Adj Close']
    
    # takes one column from dataframe
    B_MA = pd.Series((data.rolling(n, min_periods=n).mean()), name='B_MA')
    sigma = data.rolling(n, min_periods=n).std() 
    
    BU = pd.Series((B_MA + m * sigma), name='BU')
    BL = pd.Series((B_MA - m * sigma), name='BL')
    
    df = df.join(B_MA)
    df = df.join(BU)
    df = df.join(BL)
    
    return df


def send_mail(subjectline, body, to_addr):

    host = "smtp.tom.com"
    port = 25
    from_addr = "risson_chanlun@tom.com"
    sender_email = "risson_chanlun@tom.com"
    paaa = "***"

    message = MIMEMultipart()
    message.attach(MIMEText(body, "html"))
    message['Subject'] = subjectline
    message['From'] = from_addr
    message['To'] = to_addr
    msg_body = message.as_string()
   
    with SMTP(host, port) as server:
        server.login(sender_email, paaa)
        server.sendmail(message['From'], message['To'], msg_body)

    print("Sent!")

In [5]:
ticktock =  0
while ticktock<24:

    api = TdxHq_API()
    host = hq_hosts[10]
    n = 480   # datapoint rolling window
    m = 2    # sigma width

    df_hyao = []
    boll_hyao = []
    df_lan = []
    boll_lan = []

    if api.connect(host[1], host[2]):
        print('Connected!')

        for stock in stocks_hyao:
            df = api.to_df(api.get_security_bars(0, stock[0], stock[1], 0, 500)) # 5 min k lines
            df_hyao += [df]

        for stock in stocks_lan:
            df = api.to_df(api.get_security_bars(0, stock[0], stock[1], 0, 500)) # 5 min k lines
            df_lan += [df]

        api.disconnect()

    else:   
        send_mail("布林程序连接数据失败","布林监控程序连接通达信数据失败","270156760@qq.com")

    for df in df_hyao:
        boll_hyao += [bollinger_bands(df, n, m)]
    
    for df in df_lan:
        boll_lan += [bollinger_bands(df, n, m)]
        
    for i in range(len(df_hyao)):
        boll = boll_hyao[i]
        if boll.iloc[-2]['high'] < boll.iloc[-2]['BU'] and boll.iloc[-1]['high'] >= boll.iloc[-2]['BU']:
            stock_nm = stocks_hyao[i][2]
            subject = f"[{stock_nm}] 5F布林上穿上轨"
            body = f"时间：{boll.iloc[-1]['datetime']}，现价：{boll.iloc[-1]['close']}"
            send_mail(subject,body,"270156760@qq.com")
        if (boll.iloc[-3]['low'] > boll.iloc[-3]['BU'] or boll.iloc[-2]['low'] > boll.iloc[-2]['BU']) and boll.iloc[-1]['low'] <= boll.iloc[-2]['BU']:
            stock_nm = stocks_hyao[i][2]
            subject = f"[{stock_nm}] 5F布林下穿上轨"
            body = f"时间：{boll.iloc[-1]['datetime']}，现价：{boll.iloc[-1]['close']}"
            send_mail(subject,body,"270156760@qq.com")

    for i in range(len(df_lan)):
        boll = boll_lan[i]
        if boll.iloc[-2]['high'] < boll.iloc[-2]['BU'] and boll.iloc[-1]['high'] >= boll.iloc[-2]['BU']:
            stock_nm = stocks_lan[i][2]
            subject = f"[{stock_nm}] 5F布林上穿上轨"
            body = f"时间：{boll.iloc[-1]['datetime']}，现价：{boll.iloc[-1]['close']}"
            send_mail(subject,body,"6195119@qq.com")
        if (boll.iloc[-3]['low'] > boll.iloc[-3]['BU'] or boll.iloc[-2]['low'] > boll.iloc[-2]['BU']) and boll.iloc[-1]['low'] <= boll.iloc[-2]['BU']:
            stock_nm = stocks_lan[i][2]
            subject = f"[{stock_nm}] 5F布林下穿上轨"
            body = f"时间：{boll.iloc[-1]['datetime']}，现价：{boll.iloc[-1]['close']}"
            send_mail(subject,body,"6195119@qq.com")

    ticktock += 1
    time.sleep(300)
    

Connected!
Connected!
Connected!
Connected!
Connected!
Connected!
Connected!


KeyboardInterrupt: 